<a href="https://colab.research.google.com/github/hugotomita1201/yachay.ai_project/blob/main/distilbert_classification_100_targets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

# %%
import torch
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer
from transformers import BertConfig
import pandas as pd
import csv
import time
import pickle
from torch.optim import AdamW


import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

#use gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('grouped_100_classification.csv')

# Load pre-trained model tokenizer (vocabulary)
model_name = 'distilbert-base-uncased'

config = BertConfig.from_pretrained(model_name)
config.num_labels = 100  # number of regions for classification
bert_model = AutoModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(
    model_name, normalize=True)

#bert_model = RobertaModel.from_pretrained(model_name, config=config)
#tokenizer = BertweetTokenizer.from_pretrained(model_name)

# preprocess data with function

def haversine_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371  # Earth radius in km
    pi = torch.tensor(3.141592653589793, dtype=torch.float)

    lat1, lon1, lat2, lon2 = [x * (pi / 180) for x in [lat1, lon1, lat2, lon2]]

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = torch.sin(dlat / 2) ** 2 + torch.cos(lat1) * \
        torch.cos(lat2) * torch.sin(dlon / 2) ** 2
    c = 2 * torch.atan2(torch.sqrt(a), torch.sqrt(1 - a))

    distance = earth_radius * c
    return distance


class HaversineLoss(nn.Module):
    def __init__(self):
        super(HaversineLoss, self).__init__()

    def forward(self, y_pred, y_true):
        lat1, lon1 = y_pred[:, 0], y_pred[:, 1]
        lat2, lon2 = y_true[:, 0], y_true[:, 1]

        distance = haversine_distance(lat1, lon1, lat2, lon2)
        loss = distance.mean()
        return loss


def preprocess_data(csv_file, df):

    # Filter and sample data
    df = df[df['language'] == 0]

    # Split into train and test sets
    train_features, test_features, train_labels, test_labels = train_test_split(
        df['text'], df[['group']], test_size=0.2, random_state=42)
    train_featuresx, test_featuresx, train_labelsx, test_labels_centroids = train_test_split(
        df['text'], df[['group_centroid_lat', 'group_centroid_lng']], test_size=0.2, random_state=42)
    

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
        "distilbert-base-uncased", normalize=True)

    # Tokenize the training and test text features
    train_encodings = tokenizer(train_features.tolist(), padding=True, truncation=True, return_tensors='pt')
    test_encodings = tokenizer(test_features.tolist(
    ), padding=True, truncation=True, return_tensors='pt')

    # Tokenize the training and test text features
    train_input_ids = train_encodings['input_ids'].to(device)
    test_input_ids = test_encodings['input_ids'].to(device)

    # make attention masks to let know which tokens are real and which are padding
    train_attention_mask = train_encodings['attention_mask'].to(device)
    test_attention_mask = test_encodings['attention_mask'].to(device)

    # Convert labels to PyTorch tensors with long datatype
    train_labels = torch.tensor(
        train_labels['group'].values, dtype=torch.long).to(device)
    test_labels = torch.tensor(
        test_labels['group'].values, dtype=torch.long).to(device)

    return train_encodings, test_encodings, train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels, test_labels_centroids


start_time = time.time()
# call on function
train_encodings, test_encodings, train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels, test_labels_centroids = preprocess_data(
    'grouped_dataset.csv', df)
end_time = time.time()
print(f"Preprocessing took {end_time - start_time} seconds")
# print out the shape of the data

print(train_input_ids.shape)
print(test_input_ids.shape)

# Define the BERT-based classification model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model, num_regions):
        super(BERTClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(0.2)
        self.classifier = nn.Linear(config.hidden_size, num_regions)

    def forward(self, input_ids, attention_mask):
        bert_outputs = self.bert(
            input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_outputs[0][:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits


num_regions = 100

# Initialize the BERT-based classification model
bert_classifier = BERTClassifier(bert_model, num_regions).to(device)

# Define the optimizer and the loss function
optimizer = AdamW(bert_classifier.parameters(), lr=1e-4, weight_decay=1e-7)
loss_fn = nn.CrossEntropyLoss()

# %%
# Train the model
num_epochs = 4
batch_size = 16

# set a timer to see how long it takes to train the model

start_time = time.time()
# print starting time
print(f"Training started at {time.ctime(start_time)}")
for epoch in range(num_epochs):
    for i in range(0, len(train_input_ids), batch_size):
        input_ids_batch = train_input_ids[i:i+batch_size].to(device)
        attention_mask_batch = train_attention_mask[i:i+batch_size].to(device)
        region_ids_batch = train_labels[i:i+batch_size].to(device)

        bert_classifier.zero_grad()

        logits = bert_classifier(
            input_ids_batch, attention_mask_batch).to(device)
        loss = loss_fn(logits, region_ids_batch)

        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set after each epoch
    test_batch_size = 32 

    # Evaluate the model on the test set after each epoch
    with torch.no_grad():
      test_losses = []
      test_accuracies = []
      test_distances = []

      for i in range(0, len(test_input_ids), test_batch_size):
        test_input_ids_batch = test_input_ids[i:i+test_batch_size].to(device)
        test_attention_mask_batch = test_attention_mask[i:i+test_batch_size].to(device)
        test_labels_batch = test_labels[i:i+test_batch_size].to(device)

        test_logits_batch = bert_classifier(test_input_ids_batch, test_attention_mask_batch)
        test_loss_batch = loss_fn(test_logits_batch, test_labels_batch)
        test_accuracy_batch = (test_logits_batch.argmax(dim=1) == test_labels_batch).float().mean()

        test_losses.append(test_loss_batch.item())
        test_accuracies.append(test_accuracy_batch.item())

        predicted_labels = test_logits_batch.argmax(dim=1)

      avg_test_loss = sum(test_losses) / len(test_losses)
      avg_test_accuracy = sum(test_accuracies) / len(test_accuracies)
      print(
            f"Epoch {epoch+1}: Test loss={avg_test_loss:.4f}, Test accuracy={avg_test_accuracy:.4f}")

end_time = time.time()
print(f"Training took {end_time - start_time} seconds")

predicted_labels = torch.argmax(logits, dim=1).tolist()

predicted_label_indices = [df[df['group'] == label].index[0] for label in predicted_labels]

# Get the centroid latitude and longitude for the predicted labels
predicted_lat_lng = df.loc[predicted_label_indices, ['group_centroid_lat', 'group_centroid_lng']].values

# Get the centroid latitude and longitude for the true labels
true_lat_lng = test_labels_centroids.values
# Calculate the haversine distances
test_distances = [haversine_distance(pred[0], pred[1], true[0], true[1]) for pred, true in zip(predicted_lat_lng, true_lat_lng)]

# Calculate the average distance
avg_test_distance = sum(test_distances)/ len(test_distances)
print(f"Avg distance={avg_test_distance:.4f} km")



# %%
torch.save(bert_classifier.state_dict(),
           'bert_classifier_for_grouped_data.pth')

# save to output file
output_file = 'predicted_labels_for_grouped_dataset.csv'

with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    for label in predicted_labels:
        writer.writerow([label])


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Preprocessing took 57.16174626350403 seconds
torch.Size([350036, 161])
torch.Size([87510, 129])
Training started at Tue May  2 18:50:30 2023
Epoch 1: Test loss=2.4291, Test accuracy=0.4365
Epoch 2: Test loss=1.2877, Test accuracy=0.7010
Epoch 3: Test loss=1.0153, Test accuracy=0.7649
Epoch 4: Test loss=0.9330, Test accuracy=0.7851
Training took 22552.28488254547 seconds
Avg distance=2773.9146 km
